In [ ]:
%load_ext autoreload

In [ ]:
%autoreload 2

In [ ]:
from pathlib import Path

In `config`:

    config = Config(
        max_length = [256, 256]  # in tokens, list for title vs abstract
        from_pretrained = "facebook/bart-base"
        model_type = "Twin"
    )

In [ ]:
from smtag.config import config
assert config.max_length==[256, 256]
assert config.from_pretrained=='facebook/bart-base'
assert config.model_type=='Twin'

In [ ]:
config

In [ ]:
from transformers import __version__
__version__

## Extracting examples for LM

In [ ]:
from smtag.extract import ExtractorXML

In [ ]:
# ! rm -fr /data/text/oapmc_twin
# ! rm -fr /data/text/emboj_twin

In [ ]:
corpus = "/data/xml/emboj_all"
text_examples = "/data/text/emboj_twin"
xpath = [".//article-meta/title-group/article-title", ".//abstract"]
sentence_level = False
keep_xml = False
inclusion_probability = 1.0

In [ ]:
extractor_lm = ExtractorXML(
    corpus,
    destination_dir=text_examples,
    sentence_level=sentence_level,
    xpath=xpath,
    keep_xml=keep_xml,
    inclusion_probability=inclusion_probability
)

In [ ]:
extractor_lm.extract_from_corpus()

same via CLI:


```bash
python -m smtag.cli.prepro.extract /data/xml/emboj_all /data/text/emboj_twin --xpath ".//article-meta/title-group/article-title" ".//abstract"
```

```bash
python -m smtag.cli.prepro.extract /data/xml/oapmc_articles /data/text/oapmc_twin --xpath ".//article-meta/title-group/article-title" ".//abstract"
```

```bash
python -m smtag.cli.prepro.extract /data/xml/emboj_all /data/text/emboj_trivial --xpath ".//article-meta/title-group/article-title" ".//article-meta/title-group/article-title"
```

## Preparing tokenized dataset for LM

In [ ]:
from smtag.dataprep import PreparatorLM

In [ ]:
tokenized_examples = "/data/json/emboj_twin"

In [ ]:
# ! rm -fr /data/json/oapmc_twin
! rm -fr /data/json/emboj_twin

In [ ]:
prep_lm = PreparatorLM(
    text_examples,
    tokenized_examples,
    max_length=config.max_length
)

In [ ]:
prep_lm.run()

same vie CLI:
    
```bash
python -m smtag.cli.lm.dataprep /data/text/oapmc_twin /data/json/oapmc_twin
```

```bash
python -m smtag.cli.lm.dataprep /data/text/emboj_twin /data/json/emboj_twin
```

```bash
python -m smtag.cli.lm.dataprep /data/text/emboj_trivial /data/json/emboj_trivial
```

## Train LM

In [ ]:
%autoreload 2

In [ ]:
from smtag.train.train_lm import (
    train as train_lm,
    TrainingArgumentsLM
)

In [ ]:
no_cache = False  # ALWAYS CHECK THIS IN CASE OF PROBLEMS!!!
loader_path = "./smtag/loader/loader_twin.py"
data_config_name = "NOLM"  # "SEQ2SEQ" ot have also a language model
tokenizer = config.tokenizer  # tokenizer has to be the same application-wide
model_type = "Twin"
from_pretrained = config.from_pretrained

In [ ]:
tokenizer.mask_token = '<mask>'  # why is this here? maybe because in case of character-level tokenizer
# tokenizer.unk_token = '<unk>'

In [ ]:
training_args_tokcl = TrainingArgumentsLM(
    num_train_epochs = 1,
    logging_steps = 10,  # 100 or 1000 for large oapmc_twin
    per_device_train_batch_size=64,
    per_device_eval_batch_size=64,
)
training_args_tokcl

In [ ]:
train_lm(
    training_args_tokcl,
    loader_path,
    data_config_name,
    tokenized_examples,
    no_cache,
    tokenizer,
    model_type,
    from_pretrained
)

#### With CLI:

```bash
python -m smtag.cli.lm.train smtag/loader/loader_twin.py NOLM --data_dir /data/json/oapmc_twin --per_device_train_batch_size=64 --per_device_eval_batch_size=64 --logging_steps=100 --num_train_epochs=100 --save_steps=12000
```

```bash
python -m smtag.cli.lm.train smtag/loader/loader_twin.py SEQ2SEQ --data_dir /data/json/emboj_twin --per_device_train_batch_size=16 --per_device_eval_batch_size=16 --logging_steps=100 --num_train_epochs=1 --no_cache 
```

In [ ]:
d